In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import os
import pandas as pd

print(tf.__version__)

2023-09-27 09:55:14.735060: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.0


In [2]:
tf.config.list_physical_devices('GPU')

2023-09-27 09:55:16.960875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-27 09:55:16.988817: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-27 09:55:16.988927: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
df_911 = pd.read_csv('data_911.csv', index_col=0)
df_chemtrails = pd.read_csv('data_chemtrails.csv', index_col=0)
df_flatearth = pd.read_csv('data_flatearth.csv', index_col=0)
df_moonlanding = pd.read_csv('data_moonlanding.csv', index_col=0)
df_vaccines = pd.read_csv('data_vaccines.csv', index_col=0)

In [4]:
df_911['normalized_annotation'] = df_911['normalized_annotation'].apply(pd.to_numeric, errors = 'coerce').fillna(1)
df_chemtrails['normalized_annotation'] = df_chemtrails['normalized_annotation'].apply(pd.to_numeric, errors = 'coerce').fillna(1)
df_flatearth['normalized_annotation'] = df_flatearth['normalized_annotation'].apply(pd.to_numeric, errors = 'coerce').fillna(1)
df_moonlanding['normalized_annotation'] = df_moonlanding['normalized_annotation'].apply(pd.to_numeric, errors = 'coerce').fillna(1)
df_vaccines['normalized_annotation'] = df_vaccines['normalized_annotation'].apply(pd.to_numeric, errors = 'coerce').fillna(1)

In [5]:
df_911.describe()

,normalized_annotation,Num_of_Views,Likes,Dislikes,Number_of_Comments
count,436.000000,4.360000e+02,4.360000e+02,436.000000,436.000000
mean,-0.004587,4.446220e+06,7.351247e+04,2931.933486,10780.392202
std,0.479441,9.546010e+06,1.923490e+05,6123.221874,29439.428059
min,-1.000000,5.380000e+02,0.000000e+00,0.000000,0.000000
25%,0.000000,2.076780e+05,2.223500e+03,134.750000,425.000000
50%,0.000000,9.701835e+05,1.060350e+04,774.000000,2214.000000
75%,0.000000,4.743950e+06,5.134300e+04,2976.250000,9600.750000
max,1.000000,1.386139e+08,1.902168e+06,69137.000000,393469.000000


In [6]:
df_911['normalized_annotation'] = df_911['normalized_annotation'] + 1
df_chemtrails['normalized_annotation'] = df_chemtrails['normalized_annotation'] + 1
df_flatearth['normalized_annotation'] = df_flatearth['normalized_annotation'] + 1
df_moonlanding['normalized_annotation'] = df_moonlanding['normalized_annotation'] + 1
df_vaccines['normalized_annotation'] = df_vaccines['normalized_annotation'] + 1

In [7]:
df_911.describe()

,normalized_annotation,Num_of_Views,Likes,Dislikes,Number_of_Comments
count,436.000000,4.360000e+02,4.360000e+02,436.000000,436.000000
mean,0.995413,4.446220e+06,7.351247e+04,2931.933486,10780.392202
std,0.479441,9.546010e+06,1.923490e+05,6123.221874,29439.428059
min,0.000000,5.380000e+02,0.000000e+00,0.000000,0.000000
25%,1.000000,2.076780e+05,2.223500e+03,134.750000,425.000000
50%,1.000000,9.701835e+05,1.060350e+04,774.000000,2214.000000
75%,1.000000,4.743950e+06,5.134300e+04,2976.250000,9600.750000
max,2.000000,1.386139e+08,1.902168e+06,69137.000000,393469.000000


In [8]:
df_911['Captions'] = df_911['Captions'].astype('string')
df_chemtrails['Captions'] = df_chemtrails['Captions'].astype('string')
df_flatearth['Captions'] = df_flatearth['Captions'].astype('string')
df_moonlanding['Captions'] = df_moonlanding['Captions'].astype('string')
df_vaccines['Captions'] = df_vaccines['Captions'].astype('string')

In [9]:
df_911

,Description,Title,vid_url,normalized_annotation,Num_of_Views,Likes,Dislikes,Number_of_Comments,Video_ID,Captions
Topic,,,,,,,,,,
911,"22 years ago, federal agents raided the Branch...",The Shadow of Waco | Retro Report | The New Yo...,https://www.youtube.com/watch?v=hOW9AjskoOo,1,596113,5478,420,1688,hOW9AjskoOo,it happened outside Waco Texas a heavily armed...
911,:::VIEWER DISCRETION ADVISED::: Watch and list...,Former Abortionist Dr. Levatino Destroys Pro-C...,https://www.youtube.com/watch?v=dIRcw45n9RU,1,520316,15965,640,6828,dIRcw45n9RU,thanks for coming it's nice to see a good turn...
911,An investigation of how Donald Trump defied ex...,Trump's Road to the White House (full film) | ...,https://www.youtube.com/watch?v=SMwXKl0odq8,1,1543110,12403,1721,5919,SMwXKl0odq8,">> Tonight... >> I, Donald John Trump, do sol..."
911,The F.B.I.'s greatest unsolved case! Presente...,The Strange Disappearance of D.B. Cooper,https://www.youtube.com/watch?v=oHSehKtDyoI,1,12129914,207614,7949,20664,oHSehKtDyoI,"- This week on Buzzfeed Unsolved, we discuss t..."
911,▶︎ Shockingly Offensive Auditions Has Simon Co...,SHOCKINGLY OFFENSIVE AUDITIONS Have Simon Cowe...,https://www.youtube.com/watch?v=N9COy7O7K-U,1,9681212,91657,5576,18965,N9COy7O7K-U,I'm Mason noise I'm 22 and I'm from Birmingha...
...,...,...,...,...,...,...,...,...,...,...
911,NaN,Conspiracy Theories - 9/11 Truth Movement part 1,https://www.youtube.com/watch?v=MeHqsfgB2v8,1,288314,902,257,1049,MeHqsfgB2v8,the 911 truth movement is about the search for...
911,"Michael Shermer (Author, ""Skeptic"") joins Dave...",Conspiracy Theories with Michael Shermer,https://www.youtube.com/watch?v=eGGjCZbMh9Y,0,27250,616,120,433,eGGjCZbMh9Y,you do talk a lot about conspiracy theories an...
911,The collapse of the three World Trade Center s...,9/11 - Evidence of Controlled Demolition of WT...,https://www.youtube.com/watch?v=PGchJ3b6hVE,2,7981,52,3,12,PGchJ3b6hVE,okay yeah a lot of a deal material case you co...


In [10]:
from sklearn.model_selection import train_test_split

X_train_911, X_test_911, y_train_911, y_test_911 = train_test_split(
    df_911['Captions'], df_911['normalized_annotation'], test_size=0.1, shuffle=True
)

In [11]:
X_train_911

Topic
911    and I can go like this and I can do that on my...
911    >>Female Presenter: Welcome everybody. It's
 g...
911    This is Neverland, currently for sale, former ...
911    the world is full of wild conspiracy theories ...
911    DYLAN WELCH, REPORTER: Sydney grandmother
 Kar...
                             ...                        
911    MARSH IS JOINING US NOW WITH  MORE. WHAT'S THE...
911    welcome back to football daily where today we ...
911    9:11 magazine in wet mode the new 911 on an ex...
911    you don't know already see Keith is a ticket a...
911    all right how is it going today Lourdes my nam...
Name: Captions, Length: 392, dtype: string

In [12]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", trainable=False)
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=False)

2023-09-27 09:55:18.557719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-27 09:55:18.557919: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-27 09:55:18.558025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-27 09:55:19.975367: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-27 09:55:19.975490: I tensorflow/compile

In [13]:
optim = tf.keras.optimizers.legacy.Adam()

In [14]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

In [15]:
# bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# concat = tf.keras.layers.Concatenate()(outputs)

# nn layers
# l1 = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
nn_911 = tf.keras.layers.Dropout(0.1)(outputs['sequence_output'])
nn_911 = tf.keras.layers.LSTM(units=128)(nn_911)
# l4 = tf.keras.layers.LSTM(units=128)(l2)
nn_911 = tf.keras.layers.Dropout(0.1)(nn_911)
# out = tf.keras.layers.Dense(units=3, activation='softmax')(l5)
nn_911 = tf.keras.layers.Dense(units=3)(nn_911)
out_911 = tf.keras.layers.Softmax()(nn_911)

In [16]:
# CHANGE OUTPUT LAYER AFTER ADJUSTING 
model_911 = tf.keras.Model(inputs = [text_input], outputs = [out_911])

In [17]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

In [18]:
model_911.compile(
    loss=loss,
    metrics=['accuracy'],
    optimizer=optim
)

In [19]:
history = model_911.fit(X_train_911, y_train_911, epochs=50, callbacks=[early_stopping], validation_data=(X_test_911, y_test_911))

Epoch 1/50


2023-09-27 09:55:57.363035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


13/13 [==============================] - 33s 1s/step - loss: 0.7118 - accuracy: 0.7602 - val_loss: 0.6673 - val_accuracy: 0.7955
Epoch 2/50
13/13 [==============================] - 15s 1s/step - loss: 0.5094 - accuracy: 0.7985 - val_loss: 0.6323 - val_accuracy: 0.7955
Epoch 3/50
13/13 [==============================] - 15s 1s/step - loss: 0.3572 - accuracy: 0.8571 - val_loss: 0.6892 - val_accuracy: 0.7955
Epoch 4/50
13/13 [==============================] - 15s 1s/step - loss: 0.2577 - accuracy: 0.8954 - val_loss: 0.7079 - val_accuracy: 0.7955
Epoch 5/50
13/13 [==============================] - 15s 1s/step - loss: 0.1487 - accuracy: 0.9668 - val_loss: 0.8034 - val_accuracy: 0.7500
Epoch 6/50
13/13 [==============================] - 15s 1s/step - loss: 0.0912 - accuracy: 0.9770 - val_loss: 0.8555 - val_accuracy: 0.7955


In [20]:
# accuracy was .94ish
model_911.evaluate(X_test_911, y_test_911)

2/2 [==============================] - 1s 341ms/step - loss: 0.8555 - accuracy: 0.7955


[0.8555322289466858, 0.7954545617103577]

In [21]:
from sklearn.model_selection import train_test_split

X_train_chemtrails, X_test_chemtrails, y_train_chemtrails, y_test_chemtrails = train_test_split(
    df_chemtrails['Captions'], df_chemtrails['normalized_annotation'], test_size=0.1, shuffle=True
)

In [22]:
# bert layers
text_input_chemtrails = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text_chemtrails = bert_preprocess(text_input_chemtrails)
outputs_chemtrails = bert_encoder(preprocessed_text_chemtrails)

# concat = tf.keras.layers.Concatenate()(outputs)

# nn layers
# l1 = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
nn_ct = tf.keras.layers.Dropout(0.1)(outputs_chemtrails['sequence_output'])
nn_ct = tf.keras.layers.LSTM(units=128)(nn_ct)
# l4 = tf.keras.layers.LSTM(units=128)(l2)
nn_ct = tf.keras.layers.Dropout(0.1)(nn_ct)
nn_ct = tf.keras.layers.Dense(units=3)(nn_ct)
out_ct = tf.keras.layers.Softmax()(nn_ct)

In [23]:
model_chemtrails = tf.keras.Model(inputs = [text_input_chemtrails], outputs = [out_ct])

In [24]:
model_chemtrails.compile(
    loss=loss,
    optimizer=optim,
    metrics=['accuracy']
)

In [25]:
history_ct = model_chemtrails.fit(X_train_chemtrails, y_train_chemtrails, epochs=50, callbacks=[early_stopping], validation_data=(X_test_chemtrails, y_test_chemtrails))

Epoch 1/50


14/14 [==============================] - 25s 1s/step - loss: 0.8342 - accuracy: 0.6000 - val_loss: 0.8402 - val_accuracy: 0.6122
Epoch 2/50
14/14 [==============================] - 17s 1s/step - loss: 0.5751 - accuracy: 0.8046 - val_loss: 0.7530 - val_accuracy: 0.6531
Epoch 3/50
14/14 [==============================] - 17s 1s/step - loss: 0.4655 - accuracy: 0.8230 - val_loss: 0.8437 - val_accuracy: 0.6327
Epoch 4/50
14/14 [==============================] - 17s 1s/step - loss: 0.3111 - accuracy: 0.8851 - val_loss: 0.8660 - val_accuracy: 0.6939
Epoch 5/50
14/14 [==============================] - 17s 1s/step - loss: 0.1395 - accuracy: 0.9609 - val_loss: 1.0259 - val_accuracy: 0.7347
Epoch 6/50
14/14 [==============================] - 17s 1s/step - loss: 0.0759 - accuracy: 0.9839 - val_loss: 1.0522 - val_accuracy: 0.6939


In [26]:
model_chemtrails.evaluate(X_test_chemtrails, y_test_chemtrails)

2/2 [==============================] - 2s 471ms/step - loss: 1.0522 - accuracy: 0.6939


[1.0522459745407104, 0.6938775777816772]

In [27]:
[0.7436542510986328, 0.767123281955719]

[0.7436542510986328, 0.767123281955719]

In [28]:
from sklearn.model_selection import train_test_split

X_train_flatearth, X_test_flatearth, y_train_flatearth, y_test_flatearth = train_test_split(
    df_flatearth['Captions'], df_flatearth['normalized_annotation'], test_size=0.1, shuffle=True
)


In [29]:
# bert layers
text_input_flatearth = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text_flatearth = bert_preprocess(text_input_flatearth)
outputs_flatearth = bert_encoder(preprocessed_text_flatearth)

# concat = tf.keras.layers.Concatenate()(outputs)

# nn layers
# l1 = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
nn_fe = tf.keras.layers.Dropout(0.1)(outputs_flatearth['sequence_output'])
nn_fe = tf.keras.layers.LSTM(units=128)(nn_fe)
# l4 = tf.keras.layers.LSTM(units=128)(l2)
nn_fe = tf.keras.layers.Dropout(0.1)(nn_fe)
nn_fe = tf.keras.layers.Dense(units=3)(nn_fe)
out_fe = tf.keras.layers.Softmax()(nn_fe)


In [30]:
model_fe = tf.keras.Model(inputs = [text_input_flatearth], outputs = [out_fe])


In [31]:

model_fe.compile(
    loss=loss,
    optimizer=optim,
    metrics=['accuracy']
)

In [32]:
history_fe = model_fe.fit(X_train_flatearth, y_train_flatearth, epochs=50, callbacks=[early_stopping], validation_data=(X_test_flatearth, y_test_flatearth))


Epoch 1/50
9/9 [==============================] - 21s 2s/step - loss: 0.8852 - accuracy: 0.6140 - val_loss: 0.7386 - val_accuracy: 0.7500
Epoch 2/50
9/9 [==============================] - 12s 1s/step - loss: 0.6903 - accuracy: 0.7404 - val_loss: 0.6724 - val_accuracy: 0.7500
Epoch 3/50
9/9 [==============================] - 12s 1s/step - loss: 0.6002 - accuracy: 0.7439 - val_loss: 0.6494 - val_accuracy: 0.7500
Epoch 4/50
9/9 [==============================] - 12s 1s/step - loss: 0.4862 - accuracy: 0.8105 - val_loss: 0.7292 - val_accuracy: 0.7500
Epoch 5/50
9/9 [==============================] - 12s 1s/step - loss: 0.3810 - accuracy: 0.8526 - val_loss: 0.8052 - val_accuracy: 0.7500
Epoch 6/50
9/9 [==============================] - 12s 1s/step - loss: 0.2909 - accuracy: 0.8912 - val_loss: 0.8090 - val_accuracy: 0.7500
Epoch 7/50
9/9 [==============================] - 12s 1s/step - loss: 0.1800 - accuracy: 0.9333 - val_loss: 0.8678 - val_accuracy: 0.6875


In [33]:
model_chemtrails.evaluate(X_test_flatearth, y_test_flatearth)

1/1 [==============================] - 1s 1s/step - loss: 1.2299 - accuracy: 0.7188


[1.2298762798309326, 0.71875]

In [34]:
df_vaccines['Captions']

Topic
vaccines    oh here's some news tucker carlson can eat who...
vaccines    how protective do we really need to be of our ...
vaccines    You may not know it, but your body is engaged
...
vaccines    In the deserts of the American Southwest, spad...
vaccines    who loves my new dog I got him today after my ...
                                  ...                        
vaccines    at this Minneapolis Medical Clinic signs in En...
vaccines    >>> SHOULD PARENTS HAVE THEIR   
 CHILDREN VAC...
vaccines    RECEIVE AND WHEN THEY SHOULD GET THEM.  >> Rep...
vaccines    starting today kids in Oregon can be sent home...
vaccines    (lively upbeat music) - The effectiveness of t...
Name: Captions, Length: 621, dtype: string

In [35]:
X_train_vc, X_test_vc, y_train_vc, y_test_vc = train_test_split(
    df_vaccines['Captions'], df_vaccines['normalized_annotation'], test_size=0.1, shuffle=True
)

In [36]:
# bert layers
text_input_vc = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text_vc = bert_preprocess(text_input_vc)
outputs_vc = bert_encoder(preprocessed_text_vc)

# concat = tf.keras.layers.Concatenate()(outputs)

# nn layers
# l1 = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
nn_vc = tf.keras.layers.Dropout(0.1)(outputs_vc['sequence_output'])
nn_vc = tf.keras.layers.LSTM(units=128)(nn_vc)
# l4 = tf.keras.layers.LSTM(units=128)(l2)
nn_vc = tf.keras.layers.Dropout(0.1)(nn_vc)
nn_vc = tf.keras.layers.Dense(units=3)(nn_vc)
out_vc = tf.keras.layers.Softmax()(nn_vc)

In [37]:
model_vc = tf.keras.Model(inputs = [text_input_vc], outputs = [out_vc])

In [38]:
model_vc.compile(
    loss=loss,
    optimizer=optim,
    metrics=['accuracy']
)

In [39]:
history_vc = model_vc.fit(X_train_vc, y_train_vc, epochs=50, callbacks=[early_stopping], validation_data=(X_test_vc, y_test_vc))


Epoch 1/50


18/18 [==============================] - 31s 1s/step - loss: 0.7563 - accuracy: 0.6918 - val_loss: 0.6440 - val_accuracy: 0.6825
Epoch 2/50
18/18 [==============================] - 21s 1s/step - loss: 0.5663 - accuracy: 0.7778 - val_loss: 0.5883 - val_accuracy: 0.7460
Epoch 3/50
18/18 [==============================] - 21s 1s/step - loss: 0.4310 - accuracy: 0.8387 - val_loss: 0.5713 - val_accuracy: 0.7937
Epoch 4/50
18/18 [==============================] - 21s 1s/step - loss: 0.2915 - accuracy: 0.8907 - val_loss: 0.6906 - val_accuracy: 0.7460
Epoch 5/50
18/18 [==============================] - 21s 1s/step - loss: 0.2049 - accuracy: 0.9337 - val_loss: 0.6513 - val_accuracy: 0.8095
Epoch 6/50
18/18 [==============================] - 21s 1s/step - loss: 0.0895 - accuracy: 0.9767 - val_loss: 0.6961 - val_accuracy: 0.7619
Epoch 7/50
18/18 [==============================] - 21s 1s/step - loss: 0.0435 - accuracy: 0.9928 - val_loss: 0.7358 - val_accuracy: 0.7937


In [40]:
# TODO still make model with moonlanding
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(
    df_moonlanding['Captions'], df_moonlanding['normalized_annotation'], test_size=0.1, shuffle=True
)

In [41]:
# bert layers
text_input_ml = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text_ml = bert_preprocess(text_input_ml)
outputs_ml = bert_encoder(preprocessed_text_ml)

# concat = tf.keras.layers.Concatenate()(outputs)

# nn layers
# l1 = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
nn_ml = tf.keras.layers.Dropout(0.1)(outputs_ml['sequence_output'])
nn_ml = tf.keras.layers.LSTM(units=128)(nn_ml)
# l4 = tf.keras.layers.LSTM(units=128)(l2)
nn_ml = tf.keras.layers.Dropout(0.1)(nn_ml)
nn_ml = tf.keras.layers.Dense(units=3)(nn_ml)
out_ml = tf.keras.layers.Softmax()(nn_ml)

In [42]:
model_ml = tf.keras.Model(inputs = [text_input_ml], outputs = [out_ml])

In [43]:
model_ml.compile(
    loss=loss,
    optimizer=optim,
    metrics=['accuracy']
)

In [44]:
history_ml = model_ml.fit(X_train_ml, y_train_ml, epochs=50, callbacks=[early_stopping], validation_data=(X_test_ml, y_test_ml))

Epoch 1/50
9/9 [==============================] - 18s 1s/step - loss: 0.9060 - accuracy: 0.6351 - val_loss: 0.6098 - val_accuracy: 0.8125
Epoch 2/50
9/9 [==============================] - 10s 1s/step - loss: 0.7022 - accuracy: 0.7053 - val_loss: 0.5719 - val_accuracy: 0.8125
Epoch 3/50
9/9 [==============================] - 10s 1s/step - loss: 0.6067 - accuracy: 0.7614 - val_loss: 0.5318 - val_accuracy: 0.7500
Epoch 4/50
9/9 [==============================] - 10s 1s/step - loss: 0.5242 - accuracy: 0.7895 - val_loss: 0.5572 - val_accuracy: 0.8125
Epoch 5/50
9/9 [==============================] - 10s 1s/step - loss: 0.4427 - accuracy: 0.8035 - val_loss: 0.6681 - val_accuracy: 0.7812
Epoch 6/50
9/9 [==============================] - 10s 1s/step - loss: 0.3217 - accuracy: 0.8596 - val_loss: 0.6161 - val_accuracy: 0.7812
Epoch 7/50
9/9 [==============================] - 10s 1s/step - loss: 0.2277 - accuracy: 0.9263 - val_loss: 0.6455 - val_accuracy: 0.7812


In [45]:
# concat model

input_cc = tf.keras.layers.Input(shape=(), dtype=tf.string)
model_911_layer = model_911(input_cc)
model_chemtrails_layer = model_chemtrails(input_cc)
model_fe_layer = model_fe(input_cc)
model_vc_layer = model_vc(input_cc)
model_ml_layer = model_ml(input_cc)

nn_cc = tf.keras.layers.Concatenate()([model_911_layer, model_chemtrails_layer, model_fe_layer, model_vc_layer, model_ml_layer])
nn_cc = tf.keras.layers.Dense(units=3)(nn_cc)
out_cc = tf.keras.layers.Softmax()(nn_cc)

In [46]:
model_concat = tf.keras.Model(inputs = [input_cc], outputs = [out_cc])
model_concat.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 model (Functional)          (None, 3)                    1099418   ['input_6[0][0]']             
                                                          92                                      
                                                                                                  
 model_1 (Functional)        (None, 3)                    1099418   ['input_6[0][0]']             
                                                          92                                      
                                                                                            

In [47]:
df_combined = pd.read_csv('finished_df.csv', index_col=0)
df_combined.head()

,annotation,stance
0,ten years after 9/11 a new report out just th...,0
1,[Music] it happened outside Waco Texas a heav...,0
2,thanks for coming it's nice to see a good tur...,0
3,"&gt;&gt; Tonight... &gt;&gt; I, Donald John T...",0
4,"- This week on Buzzfeed Unsolved, we discuss ...",0


In [48]:
df_combined['numerical_stance'] = df_combined['stance'].apply(pd.to_numeric, errors = 'coerce').fillna(1)
df_combined['annotation'] = df_combined['annotation'].astype('string')

In [49]:
df_combined['annotation']

0        ten years after 9/11 a new report out just th...
1        [Music] it happened outside Waco Texas a heav...
2        thanks for coming it's nice to see a good tur...
3        &gt;&gt; Tonight... &gt;&gt; I, Donald John T...
4        - This week on Buzzfeed Unsolved, we discuss ...
                              ...                        
2667     dr. david groves is a physicist in the UK and...
2668     - Hi, I'm Matt. And I don't believe we landed...
2669                                                 <NA>
2670     hi everybody what i'd like to talk about for ...
2671     Apparently, there's an organization called "N...
Name: annotation, Length: 2672, dtype: string

In [50]:
df_combined['stance'] = df_combined['stance'] + 1
df_combined['stance']

0       1
1       1
2       1
3       1
4       1
       ..
2667    2
2668    2
2669    0
2670    1
2671    0
Name: stance, Length: 2672, dtype: int64

In [51]:
df_combined = df_combined.dropna()

In [52]:
X_train_cc, X_test_cc, y_train_cc, y_test_cc = train_test_split(
    df_combined['annotation'], df_combined['stance'], test_size=0.15, shuffle=True
)

In [53]:
type(X_train_cc)

pandas.core.series.Series

In [54]:
type(y_train_cc)

pandas.core.series.Series

In [55]:
model_concat.compile(
    loss=loss,
    optimizer=optim,
    metrics='accuracy'
)

In [56]:
history_cc = model_concat.fit(X_train_cc,  y_train_cc, epochs=50, callbacks=[early_stopping], validation_data=(X_test_cc, y_test_cc))

Epoch 1/50


71/71 [==============================] - 372s 5s/step - loss: 0.7380 - accuracy: 0.7390 - val_loss: 0.6988 - val_accuracy: 0.7350
Epoch 2/50
71/71 [==============================] - 342s 5s/step - loss: 0.6251 - accuracy: 0.7540 - val_loss: 0.6236 - val_accuracy: 0.7525
Epoch 3/50
71/71 [==============================] - 342s 5s/step - loss: 0.5551 - accuracy: 0.7787 - val_loss: 0.6346 - val_accuracy: 0.7625
Epoch 4/50
71/71 [==============================] - 348s 5s/step - loss: 0.4951 - accuracy: 0.8118 - val_loss: 0.5474 - val_accuracy: 0.7900
Epoch 5/50
71/71 [==============================] - 340s 5s/step - loss: 0.4303 - accuracy: 0.8405 - val_loss: 0.4996 - val_accuracy: 0.8125
Epoch 6/50
71/71 [==============================] - 342s 5s/step - loss: 0.3361 - accuracy: 0.8962 - val_loss: 0.4824 - val_accuracy: 0.8225
Epoch 7/50
71/71 [==============================] - 341s 5s/step - loss: 0.2807 - accuracy: 0.9117 - val_loss: 0.4451 - val_accuracy: 0.8425
Epoch 8/50
71/71 [======

In [57]:
# model_concat.save('trained_complete_withml', save_format='tf')

INFO:tensorflow:Assets written to: trained_complete_withml/assets


INFO:tensorflow:Assets written to: trained_complete_withml/assets


In [60]:
pd.DataFrame.from_dict(history_cc.history).to_csv('history_complete_withml.csv',index=False)